In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from PIL import Image
import torch

In [2]:
# pick your model
model_id = "Qwen/Qwen2-VL-2B-Instruct"

# load model + processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

...

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [9]:
# load example image
image = Image.open(r"C:\Users\User\Downloads\kiario_good.jpg")

# system + user messages
messages = [
    {
        "role": "system",
        "content": "<image>\nYou are an InDrive car condition reviewer. "
                   "Respond in a short, friendly sentence of up to 15 words. "
                   "Only perfectly clean and undamaged cars are allowed. "
                   "If the car is perfect, say something positive and end with 'Good for customers'. "
                   "If the car has any issue, respond with 'Not acceptable (reason)', "
                   "where reason is a brief description of the main problem (e.g., dirty, dent, rust). "
                   "Always make the response polite and clear."
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image}
        ]
    }
]

# build chat template
text_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# process inputs (directly pass images to processor)
inputs = processor(
    text=[text_prompt],
    images=[image],
    padding=True,
    return_tensors="pt"
).to(model.device)

# generate output
output_ids = model.generate(**inputs, max_new_tokens=50)
output_text = processor.batch_decode(output_ids, skip_special_tokens=True)[0]

print(output_text)

system
<image>
You are an InDrive car condition reviewer. Respond in a short, friendly sentence of up to 15 words. Only perfectly clean and undamaged cars are allowed. If the car is perfect, say something positive and end with 'Good for customers'. If the car has any issue, respond with 'Not acceptable (reason)', where reason is a brief description of the main problem (e.g., dirty, dent, rust). Always make the response polite and clear.
user

assistant
Good for customers. The car is clean and undamaged, with a well-maintained exterior.
